In [1]:
from PIL import Image
import rasterio
import numpy as np
import glob
import os

In [2]:
fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip/2020/'
fp_out = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/'

In [3]:
if not os.path.exists(fp_out):
    os.mkdir(fp_out)

if not os.path.exists(os.path.join(fp_out, 'raw_tif')):
    os.mkdir(os.path.join(fp_out, 'raw_tif'))

if not os.path.exists(os.path.join(fp_out, 'images')):
    os.mkdir(os.path.join(fp_out, 'images'))

if not os.path.exists(os.path.join(fp_out, 'mask')):
    os.mkdir(os.path.join(fp_out, 'mask'))

In [4]:
def save_tif_npy(numpyImage, fn_out):
    # save to both npy
    np.save(os.path.join(fp_out, 'images', f'{fn_out}.npy'),
            numpyImage)

In [5]:
for i in glob.glob(f'{fp}*'):
    if 'tif' in i:
        tif_name = i.split('/')[-1].split('.')
        im = Image.open(i)
        numpyImage = np.array(im)
        row_min = 0
        row_max = 512
        count = 0
        while row_min < numpyImage.shape[0]:
            col_min = 0
            col_max = 512
            while col_min < numpyImage.shape[1]:
                fn_out = f'{tif_name[0]}_{count}'
                if row_max > numpyImage.shape[0]:
                    if col_max > numpyImage.shape[1]:
                        save_tif_npy(numpyImage[row_min:, col_min:, :], fn_out)
                        count += 1
                    else:
                        save_tif_npy(numpyImage[row_min:, col_min:col_max, :], fn_out)
                        count += 1
                else:
                    if col_max > numpyImage.shape[1]:
                        save_tif_npy(numpyImage[row_min:row_max, col_min:, :], fn_out)
                        count += 1
                    else:
                        save_tif_npy(numpyImage[row_min:row_max, col_min:col_max, :], fn_out)
                        count += 1
                col_min = col_max
                col_max += 512
            row_min = row_max
            row_max += 512

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (124364500 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (123893600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (123770200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (124588800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (124017000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS 

In [6]:
# SPLIT TIF
import os
from itertools import product
import rasterio as rio
from rasterio import windows


in_path = '/oak/stanford/groups/deho/building_compliance/san_jose_naip/2020/'

out_path = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif/'

def get_tiles(ds, width=512, height=512):
    nols, nrows = ds.meta['width'], ds.meta['height']
#     print(nols, nrows)
    offsets = product(range(0, nrows, height), range(0, nols, width))
    offsets = [(i[1], i[0]) for i in offsets]
#     print(offsets)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

for i in glob.glob(f'{fp}*'):
    if 'tif' in i:
        input_filename = i.split('/')[-1].split('.')
        fn_out = '{}_{}.tif'
    
        with rio.open(i) as inds:
            tile_width, tile_height = 512, 512

            meta = inds.meta.copy()
            
            count = 0
            for window, transform in get_tiles(inds):
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path,fn_out.format(input_filename[0], count))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(inds.read(window=window))
                count += 1

## Generate masks from OSMBuildings

In [3]:
fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif/'
fp_out = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/mask/'

In [4]:
from shapely.geometry import box
import pyproj
from shapely.ops import transform
import geopandas as gpd
from shapely.ops import unary_union
import shapely
from rasterio.mask import mask
import pickle

Open OSMBuildings shapefile

In [5]:
len(os.listdir(fp))

16000

In [6]:
len(os.listdir('/oak/stanford/groups/deho/building_compliance/san_jose_naip/2020/'))

33

In [7]:
done = os.listdir(fp_out)

In [8]:
len(done)

4736

In [9]:
with open('no_buildings.p', 'rb') as f:
    no_buildings = pickle.load(f)

In [10]:
len(no_buildings)

3611

In [12]:
with open('no_buildings.txt', 'r') as f:
    no_buildings1 = f.readlines()
    no_buildings1 = [line.rstrip() for line in no_buildings1]

In [14]:
len(no_buildings1)

7653

In [ ]:
# no_buildings = []
count = 0
for i in glob.glob(f'{fp}*'):
    fn = i.split('/')[-1].split('.')[0]
#     if f'{fn}.npy' in done:
#         print(fn)
    if fn in no_buildings or f'{fn}.npy' in done:
        pass
    else:
        with rasterio.open(i) as inds:
            bounds = inds.bounds
            geom = box(*bounds)
            wgs84 = pyproj.CRS('EPSG:26910')
            utm = pyproj.CRS('EPSG:4326')

            project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
            utm_point = shapely.ops.transform(project, geom)

            df = gpd.read_file('../../all_buildings/data/input/OSM/san_jose.geojson', mask=utm_point)
            if len(df) == 0:
                no_buildings.append(fn)
            else:
                df = df.to_crs('epsg:26910')
                mask_im, trans = rasterio.mask.mask(inds, list(df['geometry']))
                mask_im = np.rollaxis(mask_im, 0, 3)

                mask = np.zeros((mask_im.shape[0], mask_im.shape[1]))
                mask[np.sum(mask_im == 0, axis=2) < 4] = 1 # need to check that this is right
                np.save(os.path.join(fp_out, f'{fn}.npy'),
                    mask)

        count += 1
        if count % 100 == 0:
            print(count)
            with open('no_buildings.p', 'wb') as f:
                pickle.dump(no_buildings, f)


/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

100


/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

200


/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

300


/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

In [37]:
import multiprocessing as mp

all_files = glob.glob(f'{fp}*')
# fn = [i.split('/')[-1].split('.')[0] for i in all_files]
# filtered = [i for i in fn if i not in no_buildings and f'{i}.npy' not in done]
# print(len(fn))
# print(len(filtered))

In [38]:
def driver(i):
    fn = i.split('/')[-1].split('.')[0]
    if fn in no_buildings_global or f'{fn}.npy' in done_global:
        pass
    else:
        with rasterio.open(i) as inds:
            bounds = inds.bounds
            geom = box(*bounds)
            wgs84 = pyproj.CRS('EPSG:26910')
            utm = pyproj.CRS('EPSG:4326')

            project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
            utm_point = shapely.ops.transform(project, geom)

            df = gpd.read_file('../../all_buildings/data/input/OSM/san_jose.geojson', mask=utm_point)
            if len(df) == 0:
#                 no_buildings.append(fn)
                
                with open('no_buildings.txt', 'a') as f:
                    f.write(f'{fn}\n')
                
            else:
                df = df.to_crs('epsg:26910')
                mask_im, trans = rasterio.mask.mask(inds, list(df['geometry']))
                mask_im = np.rollaxis(mask_im, 0, 3)

                mask = np.zeros((mask_im.shape[0], mask_im.shape[1]))
                mask[np.sum(mask_im == 0, axis=2) < 4] = 1 # need to check that this is right
                np.save(os.path.join('/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/mask/',
                                     f'{fn}.npy'),
                    mask)

                
def make_global(no_buildings, done):
    global no_buildings_global
    global done_global
    
    no_buildings_global = no_buildings
    done_global = done


In [39]:
nprocs = mp.cpu_count()

In [40]:
p = mp.Pool(processes=nprocs, initializer=make_global, initargs=(no_buildings, done,))
p.map(driver, all_files)

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

...

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
/usr/local/lib/python3.8/dist-packages/rasterio/mask.py:189: NodataShadowWarning: The dataset's nodata attri

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# save no_buildings

In [ ]:
import pickle
with open('no_buildings.p', 'wb') as f:
    pickle.dump(no_buildings, f)